In [1]:
import os
import glob
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json

In [11]:
root_dir = "/home/islandz/Documents/Helmet"
img_dir = os.path.join(root_dir, "images")
annot_dir = os.path.join(root_dir,"annotations")
label_dir = os.path.join(root_dir,"labels")

In [5]:
data = glob.glob(os.path.join(img_dir,"*.png"))
len(data)

5000

In [6]:
train = data[:4000]
valid = data[4000:4500]
test = data[4500:]

In [9]:
with open(os.path.join(root_dir, "train.txt"), 'w') as f:
	f.write('\n'.join(train) + '\n')

with open(os.path.join(root_dir, "valid.txt"), 'w') as f:
	f.write('\n'.join(valid) + '\n')
 
with open(os.path.join(root_dir, "test.txt"), 'w') as f:
	f.write('\n'.join(test) + '\n')

In [10]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [12]:
classes = []

from tqdm import tqdm

files = glob.glob(os.path.join(annot_dir, '*.xml'))
for fil in tqdm(files):
    
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    
    result = []
    
    tree = ET.parse(fil)
    root = tree.getroot()

    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        with open(os.path.join(label_dir, f"{filename}.txt"), "w") as f:
            f.write("\n".join(result))

100%|████████████████████████████████████| 5000/5000 [00:00<00:00, 12106.03it/s]


In [13]:
classes

['helmet', 'head', 'person']

In [14]:
import yaml

yaml_data = {"names":["helmet", "head", "person"], # 클래스 이름
             "nc":3, # 클래스 수
             "path":root_dir, # root 경로
             "train":os.path.join(root_dir, "train.txt"), # train.txt 경로
             "val":os.path.join(root_dir, "valid.txt"), # valid.txt 경로
             "test":os.path.join(root_dir,"test.txt") # test.txt 경로
             }

with open(os.path.join(root_dir, "helmet.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

In [16]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -qr requirements.txt # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 14390, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 14390 (delta 24), reused 39 (delta 20), pack-reused 14342
Receiving objects: 100% (14390/14390), 13.47 MiB | 8.03 MiB/s, done.
Resolving deltas: 100% (9923/9923), done.
/home/islandz/Documents/Helmet/yolov5
Note: you may need to restart the kernel to use updated packages.


In [17]:
!python train.py --img 640 --batch 64 --epochs 20 --data /home/islandz/Documents/Helmet/helmet.yaml --device 0 --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/home/islandz/Documents/Helmet/helmet.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.2-224-g82a5585 Python-3.9.13 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24248MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=

       7/19      17.6G    0.03633    0.03301   0.003945        236        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        500       2588      0.946      0.569      0.615      0.377

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       8/19      17.6G    0.03515    0.03263   0.003705        288        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        500       2588      0.934      0.567      0.612      0.369

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       9/19      17.6G    0.03396    0.03237   0.003394        185        640: 1
                 Class     Images  Instances          P          R      mAP50   
                   all        500       2588      0.947      0.578      0.619      0.386

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      10/1

In [18]:
!python val.py --task "test" --data /home/islandz/Documents/Helmet/helmet.yaml --weights /home/islandz/Documents/Helmet/yolov5/runs/train/exp/weights/best.pt --device 0 --save-txt

val: data=/home/islandz/Documents/Helmet/helmet.yaml, weights=['/home/islandz/Documents/Helmet/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-224-g82a5585 Python-3.9.13 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24248MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/home/islandz/Documents/Helmet/test' images and labels...500 fou
test: New cache created: /home/islandz/Documents/Helmet/test.cache
                 Class     Images  Instances          P          R      mAP50   
                   all        500       2796      0.621      0.611      0.634      0.419
                helmet        500       2026      0.926      0.913   